# Text mining on relevant pages

####  Libraries used

In [24]:
import httplib
import urllib2
import random
import re
import requests
from httplib import IncompleteRead
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sys  
#reload(sys)  
#sys.setdefaultencoding('latin-1')

In [25]:
departement=['Bakel','Bambey','Bignona','Birkelane','Bounkiling','Dagana','Dakar','Diourbel','Fatick','Foundiougne',
            'Gossas','Goudiry','Goudomp','Guediawaye','Guinguineo','Kaffrine','Kanel','Kaolack',
             'Kebemer','Kedougou','Kolda','Koumpentoum','Koungheul','Linguere','Louga','Mbour',
             'Malem Hodar','Matam','Mbacke','Medina Yoro Foulah','Nioro du Rip','Oussouye',
             'Pikine','Podor','Ranerou-Ferlo','Rufisque','Saint-Louis','Salemata','Saraya',
             'Sedhiou','Tambacounda','Thies','Tivaouane','Velingara','Ziguinchor']

arrondissement = ['Almadies','Agnam Civol','Baba Garage','Bandafassi','Brassou','Bala','Bele','Boghal','Bona',
                  'Boyngel Bamba','Bamba Thialene','Barkedji','Bembou','Bonkoto','Cas-Cas','Colobane','Coki',
                  'Darou Minam','Dar Salam','Dakateli','Dagoudane','Dodji','Dakar Plateau',
                  'Diakhao','Djilor','Djibanar','Djoulacolon','Diaroume','Darou Mousty','Dianke Makha','Diende',
                  'Djibabouya','Djiredji','Fafacourou','Fissel','Fongolembi', 'Fimela',
                  'Grand Dakar','Gamadji Sare','Guediawaye','Gniby',
                  'Kataba','Kael','Kabrousse','Katakel','Karatanba','Koussanar','Keur Moussa',
                  'Keniaba','Koulor','Koumbal','Kouthiaba Wolof','Keur Momar Sarr','Keur Mboucki',
                  'Lambaye','Lour Escale','Loudia Ouoloff','Ida Mouride',
                  'Mampatim','Makacolibantang','Mouderi','Mbane','Mbadakhoune','Missira Wadine','Missirah',
                  'Meouane','Mbediene','Mabo','Medina Sabakh',
                  'Ndindy','Ndoulo','Ndiedieng','Ndande','Ndame','Niodor','Ngoye','Ndorna','Niakhene',
                  'Niaming','Niayes','Ndiaye','Niakhar','Niaguis','Notto','Nyassia',
                  'Ogo','Orkadiere','Ouadiour', 'Parcelles Assainies','Pambal','Pakour','Paoskoto','Rao','Rufisque',
                  'Sagatta','Sagatta Dioloff','Sare Bidji','Sare Coly Salle','Sindian','Sangalkam','Sagna','Sakal',
                  'Sabodala','Sibassor','Sindia','Sessene','Salde','Thienaba','Thies',
                  'Taif','Tendouck','Tenghory','Tattaguine','Thiaroye','Toubacouta','Thille Boubacar',
                  'Velingara','Wouro Sidy','Wack Ngouna','Yang Yang']


In [ ]:
df= pd.read_csv("potential_flood_events.csv",sep=";",encoding="utf-8")

## Get the story behind links

In [ ]:
df['story']=''
def append_story_todf():
    for i , url in enumerate(df.link):
        try:
            req = urllib2.Request(url.decode('utf-8'), headers={'User-Agent' : "Magic Browser"})
            page = urllib2.urlopen( req )
        except urllib2.HTTPError,e:
            continue
        except urllib2.URLError, e:
            continue
        except httplib.HTTPException, e:
            continue
        except Exception:
            continue
        mypage=page.read()
        if len(mypage)==0:
            print "the above page does not exist"
            continue
        else:
            soup = BeautifulSoup(mypage)
            #find all paragraphs from the current "page"
            all_para=[]
            components= soup.find_all('time') + soup.find_all('p')
            for para in components:
                all_para.append(para.text)
                df.loc[i,'story']= ' '.join(all_para).strip()  
append_story_todf()

### Filter relevancy:

In [ ]:
#main subject on the link +1
df['relevancy'] = [1 if re.search("(inondation|inondes)",x) is not None else 0 for x in df.link  ]
# main subject on the content story +1
df['relevancy'] += [1 if re.search("(inondation|inondes|sinistre)",x) is not None else 0 for x in df.story]
# mention of any department on content +1:
df['relevancy'] +=[ 1 if x in departement else 0 for x in df.story ]
# mention of any arrondissement on content +1:
df['relevancy'] +=[ 1 if x in arrondissement else 0 for x in df.story  ]


## Get place by departement 

In [ ]:
df['place']=''
for i, j in enumerate(df['link']):
    x= [ k for k in departement if k.lower() in j  ]
    if x!=[]:
        if len(x)==1:
            df.loc[i,'place']=str(x)
        if len(x)>1:
            df.loc[i,'place']="-".join(x)
    else:
        df.loc[i,'place']=''
df.place.unique()

## Search place in story content (not yet mature)

In [ ]:
for i, j in enumerate(df['story']):
    x= [ k for k in departement if k.lower() in j and df.loc[i,'place']==''  ]
    if x!=[]:
        if len(x)==1:
            df.loc[i,'place']=str(x)
        if len(x)>1:
            df.loc[i,'place']="-".join(x)
df.place.unique()

## Check duplicates

In [ ]:
# mean number of character of all the links
# slice based on median
#drop duplicates based on those shorten links
# but before decrease until duplicates appear
z=[len(x) for x in df.link]
median_link_index= int(round(np.median(z)))
L=len(df.link)
while L == len(df.link):
    df['link_median']= df['link'].apply(lambda x: x[:median_link_index])
    L=len(df.link_median.unique())
    median_link_index= median_link_index - 10
df=df.drop_duplicates(subset=["link_median"])
df.shape

In [ ]:
df[df.relevancy>0]

## Get date

In [ ]:
#df['date']= [ re.search("publie [a-z-0-9]",x)[:10] if re.search("publie [a-z-0-9]",x) is not None else '' for x in df.story]